# Testing

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, accuracy_score
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.tree import export_graphviz
from mpl_toolkits.mplot3d import Axes3D
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import scipy
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neighbors import KNeighborsRegressor
from itertools import combinations

In [90]:
df_salaries_uncleaned = pd.read_csv("data/Salary_Data.csv")

# drop null entries
df_salaries_cleaned = df_salaries_uncleaned.dropna()

# standardize the degree names
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("Bachelor's Degree", "Bachelor's")
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("Master's Degree", "Master's")
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("phD", "PhD")

# normalize data types
df_salaries_cleaned["Age"] = df_salaries_cleaned["Age"].astype("int")
df_salaries_cleaned["Years of Experience"] = df_salaries_cleaned["Years of Experience"].astype("int")

# drop duplicate entries
df_salaries_cleaned = df_salaries_cleaned.drop_duplicates()

# show data summary
df_salaries_cleaned.info()
df_salaries_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1786 entries, 0 to 6631
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  1786 non-null   int32  
 1   Gender               1786 non-null   object 
 2   Education Level      1786 non-null   object 
 3   Job Title            1786 non-null   object 
 4   Years of Experience  1786 non-null   int32  
 5   Salary               1786 non-null   float64
dtypes: float64(1), int32(2), object(3)
memory usage: 83.7+ KB


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5,90000.0
1,28,Female,Master's,Data Analyst,3,65000.0
2,45,Male,PhD,Senior Manager,15,150000.0
3,36,Female,Bachelor's,Sales Associate,7,60000.0
4,52,Male,Master's,Director,20,200000.0


In [91]:
# Standardize categorical features
categorical_columns = ['Gender', 'Education Level', 'Job Title']
one_hot_encoder = OneHotEncoder(sparse_output=False)
encoded_categorical = one_hot_encoder.fit_transform(df_salaries_cleaned[categorical_columns])

encoded_categorical_columns = one_hot_encoder.get_feature_names_out(categorical_columns)

encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoded_categorical_columns)

numeric_columns = ['Age', 'Years of Experience']
target_column = ['Salary']
final_df = pd.concat([
    df_salaries_cleaned[numeric_columns].reset_index(drop=True),
    encoded_categorical_df,
    df_salaries_cleaned[target_column].reset_index(drop=True)
], axis=1)

X = final_df.drop('Salary', axis=1)
y = final_df['Salary']

# Standardize the numerical features
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)
print(mean_squared_error(y_test, y_pred_linear))
print(r2_score(y_test, y_pred_linear))

knn = KNeighborsRegressor(n_neighbors=10, metric="manhattan")
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
print(mean_squared_error(y_test, y_pred_knn))
print(r2_score(y_test, y_pred_knn))

1.845578346012396e+29
-6.4796338795486675e+19
427084898.8363408
0.8500549279892244


In [93]:
pipeline = make_pipeline(
    SelectKBest(score_func=f_regression),
    KNeighborsRegressor()
)

# Define the hyperparameters grid
param_grid = {
    'selectkbest__k': [1, 2, 3, 4, 5],  # Number of features to select
    'kneighborsregressor__n_neighbors': [n for n in range(1, 30)]
}

# Initialize GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
mse = -grid_search.best_score_
print("Best Mean Squared Error (MSE):", mse)

Best Hyperparameters: {'kneighborsregressor__n_neighbors': 11, 'selectkbest__k': 5}
Best Mean Squared Error (MSE): 566639438.145195
